In [1]:
# Freqtrade Workspace

# Notebook Helpers
%load_ext autoreload
%autoreload 2
%env WANDB_SILENT=true

# Standard Imports
from pathlib import Path
from typing import List, Callable, Tuple, Any
from wandb.wandb_run import Run
from datetime import datetime, timedelta
from copy import deepcopy

import attr
import gc
import os
import wandb
import nest_asyncio
import logging
import sys
import pandas as pd
import numpy as np
import qgrid
pd.set_option('display.max_rows', 200)

# Resolve CWD
gc.collect()
nest_asyncio.apply()

while "freqtrade" not in os.listdir():
    os.chdir("..")
    if "freqtrade" in os.listdir():
        nest_asyncio.apply()
        logger = logging.getLogger("freqtrade")
        handler = logging.StreamHandler(stream=sys.stdout)
        handler.setFormatter(logging.Formatter("%(name)s - %(message)s"))
        logger.addHandler(handler)
        logger.setLevel(logging.INFO)
        logging.getLogger("distributed.utils_perf").setLevel(logging.ERROR)

# Freqtrade Imports
from freqtrade.optimize.optimize_reports import text_table_add_metrics
from freqtrade.ml.lightning import LightningModule, LightningConfig
from freqtrade.ml.trainer import TradingTrainer
from freqtrade.ml.container import LightningContainer
from freqtrade.ml import loader, lightning_utils
from freqtrade.nbtools.preset import LocalPreset, ConfigPreset, FilePreset, CloudPreset
from freqtrade.nbtools.backtest import backtest, DataLoader
from freqtrade.nbtools import plotting, configs
from freqtrade.nbtools.helper import free_mem
from freqtrade.nbtools.pairs import PAIRS_HIGHCAP_NONSTABLE

# Constants
PATH_MOUNT = Path.cwd().parent / "mount"
PATH_DATA = PATH_MOUNT / "data"
PATH_PRESETS = PATH_MOUNT / "presets"
PATH_STRATEGIES = PATH_PRESETS / ".strategy_files"
DATALOADER = DataLoader(max_n_datasets=5)

# Define custom functions
def foo() -> str:
    return "bar"

env: WANDB_SILENT=true
freqtrade.nbtools.backtest - Initialized DataLoader with 5 max datas.


In [2]:
# NOTE: Notebook Hyperopt may allowed without interacting to CLI

PAIRLIST = [
    "BTC/USDT", "ETH/USDT", "ADA/USDT", "XRP/USDT", "EOS/USDT", "XMR/USDT", "RVN/USDT", "ZEC/USDT", "NANO/USDT", "NEO/USDT",
    "BCH/USDT", "LTC/USDT", "ETC/USDT", "XLM/USDT", "MKR/USDT", "DASH/USDT", "ICX/USDT", "BNB/USDT", "LINK/USDT", "XEM/USDT",
]

PAIRLIST = PAIRS_HIGHCAP_NONSTABLE

hyperopt_data = {
    "strategy": "ML_15n30CatBoostEnsembleHS_v2",
    "timerange": "20210701-20210901",
    "spaces": "buy",
    "epochs": 200,
    # OnlyProfitHyperOptLoss, ShortTradeDurHyperOptLoss, SharpeHyperOptLoss, SortinoHyperOptLoss, Sharpe/Sortino + HyperOptLoss + Daily
    "hyperopt_loss": "OnlyProfitHyperOptLoss",
    "hyperopt_min_trades": 20,
    "hyperopt_random_state": 1,
    "hyperopt_jobs": 14,
    "pairs": ",".join(PAIRLIST),
}

args_list = []
argparse = [args_list.append(f"--{opt} {arg}") for opt, arg in hyperopt_data.items()]
args_str = " ".join(args_list)

print("> C:\\Users\\User\\.virtualenvs\\freqtrade-futures\\Scripts\\python.exe -m notebooks.hyperopter", args_str)

> C:\Users\User\.virtualenvs\freqtrade-futures\Scripts\python.exe -m notebooks.hyperopter --strategy ML_15n30CatBoostEnsembleHS_v2 --timerange 20210701-20210901 --spaces buy --epochs 200 --hyperopt_loss OnlyProfitHyperOptLoss --hyperopt_min_trades 20 --hyperopt_random_state 1 --hyperopt_jobs 14 --pairs BTC/USDT,ETH/USDT,BCH/USDT,LTC/USDT,ADA/USDT,XRP/USDT,DOT/USDT,DOGE/USDT,ZEC/USDT,NANO/USDT,UNI/USDT,LINK/USDT,MATIC/USDT,XLM/USDT,SOL/USDT,ETC/USDT,VET/USDT,THETA/USDT,NEO/USDT,EOS/USDT,TRX/USDT,FIL/USDT,XMR/USDT,AAVE/USDT,MKR/USDT,ATOM/USDT,ALGO/USDT,KSM/USDT,TFUEL/USDT,LUNA/USDT,BTT/USDT,AVAX/USDT,COMP/USDT,DASH/USDT,DCR/USDT,EGLD/USDT,WAVES/USDT,YFI/USDT,XEM/USDT,CHZ/USDT,SUSHI/USDT,HOT/USDT,ZIL/USDT,SNX/USDT,MANA/USDT,ENJ/USDT,HNT/USDT,BAT/USDT,NEAR/USDT,QTUM/USDT,ONE/USDT,ONT/USDT,BNT/USDT,ZRX/USDT,FTM/USDT,OMG/USDT,ICX/USDT,ANKR/USDT,RVN/USDT,CRV/USDT,FTT/USDT


In [4]:
""" Iterations of Freqtrade Strategy Development

1. Conceptualize the strategy
    1.1 Indicators Needed
    1.2 Buy Logic and Filters
    1.3 Sell Logic
    1.4 Money Management (ROI Table, TP, SL, TSL, Custom SL?)
    
2. Code your freqtrade version of that strategy.

3. Define the hyperopt model
    3.1 Use ROI table? Use stop loss? If use stop loss, is it trailing?
    3.2 Hyperparameters that run at on_populate_indicators() -> DON'T USE TOO MANY OF THEM!
    3.3 Hyperparameters that used at on_populate_buy_trend() and on_populate_sell_trend() -> CAN USE TOO MANY OF THEM.
    
4. Code the HyperStrategy (HS) version of that strategy.
"""

C:\Python\Python39\python.exe
C:\Users\User\AppData\Local\Microsoft\WindowsApps\python.exe
